In [1]:
!pip install flask faiss-cpu sentence-transformers mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.0/34.0 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12

In [2]:
import os
import re
import json

def load_corpus(file_path):
    """Load and clean text from a file"""
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()
    text = re.sub(r"\s+", " ", text).strip()
    return text

def chunk_text(text, chunk_size=300):
    """Split text into chunks of a given size"""
    words = text.split()
    return [" ".join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

if __name__ == "__main__":
    # Ensure the data directory exists
    os.makedirs("data", exist_ok=True)

    corpus_text = load_corpus("/content/corpus.txt")  # Ensure this file exists
    chunks = chunk_text(corpus_text)

    # Save chunks as JSON
    with open("data/chunks.json", "w", encoding="utf-8") as f:
        json.dump(chunks, f)

    print(f"Processed {len(chunks)} text chunks.")


Processed 20 text chunks.


In [3]:
import json

# Open the text file correctly
with open("/content/corpus.txt", "r", encoding="utf-8") as f:
    text_chunks = f.read().split("\n")  # Splitting text into lines

# Save the text as JSON
with open("/content/corpus.json", "w", encoding="utf-8") as f:
    json.dump(text_chunks, f)

print(f"Processed {len(text_chunks)} text chunks.")


Processed 247 text chunks.


In [4]:
!pip install faiss-cpu

In [5]:
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Load text chunks
with open("/content/corpus.json", "r", encoding="utf-8") as f:
    text_chunks = json.load(f)

# Load embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Convert text to embeddings
embeddings = model.encode(text_chunks)

# Store embeddings in FAISS
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

# Create directory if not exists
os.makedirs("vector_store", exist_ok=True)


# Save index
faiss.write_index(index, "vector_store/faiss_index.bin")

# Save metadata
with open("vector_store/metadata.json", "w", encoding="utf-8") as f:
    json.dump(text_chunks, f)

print("Embeddings stored successfully.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embeddings stored successfully.


In [6]:
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Load vector index
index = faiss.read_index("vector_store/faiss_index.bin")

# Load text metadata
with open("vector_store/metadata.json", "r", encoding="utf-8") as f:
    text_chunks = json.load(f)

# Load model
model = SentenceTransformer("all-MiniLM-L6-v2")

def retrieve_top_k(query, k=3):
    """Retrieve top-k relevant chunks based on query"""
    query_embedding = model.encode([query])
    distances, indices = index.search(np.array(query_embedding), k)
    return [text_chunks[i] for i in indices[0]]

def generate_response(query):
    """RAG response generation"""
    retrieved_chunks = retrieve_top_k(query)
    context = " ".join(retrieved_chunks)
    response = f"Based on what I found: {context}"
    return response

if __name__ == "__main__":
    user_query = input("Ask a question: ")
    print(generate_response(user_query))


Ask a question: only ai
Based on what I found: Artificial intelligence (AI) Artificial intelligence Machine learning as subfield of AI[21]


In [ ]:
!pip install pyngrok # Install pyngrok

from flask import Flask, request, jsonify
import mysql.connector
import datetime
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from pyngrok import ngrok  # Import ngrok

# ... rest of the code

app = Flask(__name__)

# MySQL connection configuration
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'Raj@1234',
    'database': 'ecom'
}

# Establish connection to the database using db_config
conn = mysql.connector.connect(**db_config)  # Create conn object using db_config
cursor = conn.cursor()  # Now use the conn object

# Create table if not exists
cursor.execute("""
CREATE TABLE IF NOT EXISTS chat_history (
    id INT AUTO_INCREMENT PRIMARY KEY,
    timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    role VARCHAR(10),
    content TEXT
)
""")
conn.commit() #Commit using conn

# ... (rest of your code) ...

# Load vector index
index = faiss.read_index("vector_store/faiss_index.bin")

# Load text metadata
with open("vector_store/metadata.json", "r", encoding="utf-8") as f:
    text_chunks = json.load(f)

# Load model
model = SentenceTransformer("all-MiniLM-L6-v2")

def retrieve_top_k(query, k=3):
    """Retrieve top-k relevant chunks based on query"""
    query_embedding = model.encode([query])
    distances, indices = index.search(np.array(query_embedding), k)
    return [text_chunks[i] for i in indices[0]]

def generate_response(query):
    """RAG response generation"""
    retrieved_chunks = retrieve_top_k(query)
    context = " ".join(retrieved_chunks)
    response = f"Based on what I found: {context}"
    return response


@app.route("/chat", methods=["POST"])
def chat():
    """Handle user query and return chatbot response"""
    user_query = request.json.get("query")
    if not user_query:
        return jsonify({"error": "No query provided"}), 400

    response = generate_response(user_query)

    # Store in MySQL
    cursor.execute("INSERT INTO chat_history (role, content) VALUES (%s, %s)", ("user", user_query))
    cursor.execute("INSERT INTO chat_history (role, content) VALUES (%s, %s)", ("system", response))
    db.commit()

    return jsonify({"response": response})

@app.route("/history", methods=["GET"])
def get_history():
    """Fetch chat history"""
    cursor.execute("SELECT * FROM chat_history ORDER BY timestamp DESC")
    history = cursor.fetchall()

    return jsonify([
        {"id": row[0], "timestamp": row[1], "role": row[2], "content": row[3]}
        for row in history
    ])

if __name__ == "__main__":
    # Create a tunnel to the Flask app
    public_url = ngrok.connect(5000)
    print(f"Flask app is running at {public_url}")
    app.run(debug=True, use_reloader=False)  # Don't use reloader in Colab
